In [3]:
import pandas as pd

# Load your dataset
df = pd.read_csv('/Users/deevit/Desktop/Challenge/data/overview monthly data/19981201 South Australia.csv')

# Create Renewables column
df['Renewables - GWh'] = (
    df['Wind -  GWh']
    + df['Solar (Utility) -  GWh']
    + df['Solar (Rooftop) -  GWh']
)

# Create Fossil Fuels column
df['Fossil Fuels - GWh'] = (
    df['Coal (Brown) -  GWh']
    + df['Distillate -  GWh']
    + df['Gas (Steam) -  GWh']
    + df['Gas (CCGT) -  GWh']
    + df['Gas (OCGT) -  GWh']
    + df['Gas (Reciprocating) -  GWh']
)

# Select only the needed columns
export_df = df[['date', 'Renewables - GWh', 'Fossil Fuels - GWh']]

# Export to CSV
export_path = 'renewables_fossilfuels_timeseries.csv'
export_df.to_csv(export_path, index=False)

export_path


'renewables_fossilfuels_timeseries.csv'

In [2]:
import pandas as pd

# --------------------------
# 1. Load dataset
# --------------------------
df = pd.read_csv('/Users/deevit/Desktop/Challenge/data/Data/overview monthly data/19981201 South Australia.csv')

# --------------------------
# 2. Compute VRE (Variable Renewable Energy)
# --------------------------
df['VRE - GWh'] = (
    df['Wind -  GWh'] +
    df['Solar (Utility) -  GWh'] +
    df['Solar (Rooftop) -  GWh']
)

# --------------------------
# 3. Compute Total Generation (Fossil + Renewables + Battery Discharge)
# --------------------------
df['Total Generation - GWh'] = (
    df['Coal (Brown) -  GWh'] +
    df['Distillate -  GWh'] +
    df['Gas (Steam) -  GWh'] +
    df['Gas (CCGT) -  GWh'] +
    df['Gas (OCGT) -  GWh'] +
    df['Gas (Reciprocating) -  GWh'] +
    df['Wind -  GWh'] +
    df['Solar (Utility) -  GWh'] +
    df['Solar (Rooftop) -  GWh'] +
    df['Battery (Discharging) -  GWh']
)

# --------------------------
# 4. Compute VRE Share (%)
# --------------------------
df['VRE Share (%)'] = (
    df['VRE - GWh'] / df['Total Generation - GWh'] * 100
)

# --------------------------
# 5. Create final export dataframe
# --------------------------
export_df = df[['date', 'VRE - GWh', 'Total Generation - GWh', 'VRE Share (%)']]

# --------------------------
# 6. Export to CSV
# --------------------------
export_df.to_csv('vre_share_timeseries.csv', index=False)

print("Saved: vre_share_timeseries.csv")


Saved: vre_share_timeseries.csv


In [12]:
#chart 4 
import pyarrow.parquet as pq

# table = pq.ParquetFile("/Users/deevit/Desktop/Challenge/data/Data/curated/sa1_2005-04_features.parquet")
# print(table.schema)


import pandas as pd
import glob
import os
import re
import pyarrow.parquet as pq

# -------------------------------------------------
# 1. Find all curated parquet files
# -------------------------------------------------
parquet_files = sorted(
    glob.glob("/Users/deevit/Desktop/Challenge/data/Data/curated/*.parquet")
)

rows = []

for file in parquet_files:
    # --- parse year & month from filename, e.g. sa1_2010-04_features.parquet
    fname = os.path.basename(file)
    m = re.search(r"(\d{4})-(\d{2})", fname)
    if m is None:
        print(f"Skipping file with unexpected name: {fname}")
        continue

    year = int(m.group(1))
    month = int(m.group(2))

    # only keep 2010–2013 for this graph
    if not (2018 <= year <= 2020):
        continue

    # --- read parquet with only needed columns
    table = pq.read_table(file, columns=["price", "hour"])
    df = table.to_pandas()

    df["year"] = year
    df["month"] = month

    rows.append(df)

# -------------------------------------------------
# 2. Combine all months/years
# -------------------------------------------------
full_df = pd.concat(rows, ignore_index=True)

# drop missing prices just in case
full_df = full_df.dropna(subset=["price"])

# -------------------------------------------------
# 3. Map month → season
# -------------------------------------------------
def month_to_season(m):
    if m in [12, 1, 2]:
        return "Summer"
    elif m in [3, 4, 5]:
        return "Autumn"
    elif m in [6, 7, 8]:
        return "Winter"
    elif m in [9, 10, 11]:
        return "Spring"

full_df["season"] = full_df["month"].apply(month_to_season)

# -------------------------------------------------
# 4. Keep only columns needed for diurnal price plot
# -------------------------------------------------
export_df = full_df[["year", "month", "season", "hour", "price"]]

# -------------------------------------------------
# 5. Save to CSV
# -------------------------------------------------
output_path = "/Users/deevit/Desktop/Challenge/data/Charts/Graph4_2010-2013/sa_market_diurnal_price_2018-2020.csv"
export_df.to_csv(output_path, index=False)

print(f"Saved: {output_path}")
print(export_df.head())


Saved: /Users/deevit/Desktop/Challenge/data/Charts/Graph4_2010-2013/sa_market_diurnal_price_2018-2020.csv
   year  month  season  hour      price
0  2018      1  Summer     0   99.00035
1  2018      1  Summer     0   93.40074
2  2018      1  Summer     0   88.53049
3  2018      1  Summer     1   96.53000
4  2018      1  Summer     1  109.30218
